# Car Insurance Analysis

Hey,
This project is a bonus material from DataCamp course. In this project, I will fundamentally answer a single question: **which variable is the best predictor of whether a costumer will put in a claim?**
According to the project's tale, this variable will be the single feature in a machine learning model, which will predict if a client will or will not make a claim on their insurance. 

Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [75]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit


In [76]:
df = pd.read_csv('car_insurance.csv')
print(df.head())
print(df.info())

       id  age  gender  ... duis past_accidents outcome
0  569520    3       0  ...    0              0     0.0
1  750365    0       1  ...    0              0     1.0
2  199901    0       0  ...    0              0     0.0
3  478866    0       1  ...    0              0     0.0
4  731664    1       1  ...    0              1     1.0

[5 rows x 18 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 n

In [77]:
cat_4 = [0,1,2,3]
cat_2 = [0,1]

cat_4_col = list(df[['age', 'driving_experience','income']].columns)

cat_2_col = list(df[['gender','vehicle_ownership', 'vehicle_year', 'married', 'vehicle_type']].columns)

def assert_col(col, cat = 4):
    global df, cat_4, cat_2
    if cat == 4:
        try:
            assert df[col].all() in cat_4
            print(f'the {col} column is clean.')
        except AssertionError as e:
            print(f'the {col} column is not clean.')
            print(e)
    else:
        try:
            assert df[col].all() in cat_2
            print(f'the {col} column is clean.')
        except AssertionError as e:
            print(f'the {col} column is not clean.')
            print(e)

def assert_cat(col):
    global df
    col_type = str(df[col].dtype)
    if col_type != 'category':
        df[col] = df[col].astype('category')
        print(f'the {col} column is not category. astype method applied.')
        return df[col]
    else:
        print(f'the {col} column is a category. Returning the col')
        return df[col]
    
for col in cat_4_col:
    assert_col(col, cat = 4)
    df[col] = assert_cat(col)
for col in cat_2_col:
    assert_col(col, cat = 2)
    df[col] = assert_cat(col)

try:
    assert df['education'].unique().all() in [0,1,2]
    print('The education column is clean and contains only valid values')
except AssertionError:
    print('The education column contains invalid or unexpected values. Please, verify.')

the age column is clean.
the age column is not category. astype method applied.
the driving_experience column is clean.
the driving_experience column is not category. astype method applied.
the income column is clean.
the income column is not category. astype method applied.
the gender column is clean.
the gender column is not category. astype method applied.
the vehicle_ownership column is clean.
the vehicle_ownership column is not category. astype method applied.
the vehicle_year column is clean.
the vehicle_year column is not category. astype method applied.
the married column is clean.
the married column is not category. astype method applied.
the vehicle_type column is clean.
the vehicle_type column is not category. astype method applied.
The education column contains invalid or unexpected values. Please, verify.


In [78]:
# Education is unique but its a string. Convert it.
df['education'].unique()

df['education'] = df['education'].astype('category')

In [79]:
# credit_score and annual mileage has NAs - more than 9% of the data frame
# substitute NAs for the mean or median

df[['credit_score', 'annual_mileage']].agg(['mean', 'median'])

print(f'As the mean and median of credit_score and annual mileage are both similar, I\'ll use mean as the substitute for NAs')


As the mean and median of credit_score and annual mileage are both similar, I'll use mean as the substitute for NAs


In [80]:
# Replacing NAs in credit_score and annual_mileage
mean_credit = df['credit_score'].mean()
mean_mileage = df['annual_mileage'].mean()

df = df.fillna({'credit_score': mean_credit,
                'annual_mileage': mean_mileage})

In [81]:
# Postal code are repetitive, so they are a category
df['postal_code'] = df['postal_code'].astype('category')

In [82]:
# scaning all the features to search the best based on accuracy
def accuracy(logit_model):
    conf_matrix = logit_model.pred_table()
    acc = (conf_matrix[0,0] + conf_matrix[1,1])/(conf_matrix[0,0] + conf_matrix[1,1] + conf_matrix[1,0] + conf_matrix[0,1])
    return acc

# listing all the features
cols_df = list(df.columns)
cols_df.remove('outcome')
cols_df.remove('id')

logits = [logit(f'outcome ~ {col}', data = df).fit() for col in cols_df]

accs = [accuracy(mdl) for mdl in logits]

features = pd.DataFrame({'feature' : cols_df,
                         'accuracy' : accs}).sort_values('accuracy', ascending=False)

print(features) # we have a winner.

Optimization terminated successfully.
         Current function value: 0.506484
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615951
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.467092
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.603742
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.531499
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.572557
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.552412
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.572668
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.586659
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595431
  

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



Optimization terminated successfully.
         Current function value: 0.558922
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.598699
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.549220
         Iterations 7
                feature  accuracy
2    driving_experience    0.7771
0                   age    0.7747
4                income    0.7425
6     vehicle_ownership    0.7351
5          credit_score    0.7054
10          postal_code    0.6987
11       annual_mileage    0.6904
1                gender    0.6867
3             education    0.6867
7          vehicle_year    0.6867
8               married    0.6867
9              children    0.6867
12         vehicle_type    0.6867
13  speeding_violations    0.6867
14                 duis    0.6867
15       past_accidents    0.6867


In [94]:
# Best feature

best_feature_df = pd.DataFrame({'best_feature': [features.iloc[0, 0]],
                             'best_accuracy': [features.iloc[0, 1]]})

best_feature = best_feature_df['best_feature'][0]
best_accuracy = round(best_feature_df['best_accuracy'][0]*100)

print(f'the best feature is {best_feature} with a accuracy of {best_accuracy}%')

the best feature is driving_experience with a accuracy of 78%
